In [1]:
import sys
import os
import numpy as np
import pandas as pd

import h5py

import gc


In [ ]:
#Load scores and auxiliary data, compute mean over folds, and save new scores

#Specify dir with score files
grad_dir = '../../../borzoi/examples/saved_models/flowfish'

fold_index = [3]
cross_index = [0, 1]

#Initialize HDF5
scores_h5 = h5py.File(grad_dir + '/scores_mean.h5', 'w')

seqs = None
grads = None
preds = None
genes = None
chrs = None
starts = None
ends = None
strands = None

rep_i = 0

#Loop over folds and crosses
for fi in fold_index :
    for ci in cross_index :

        print("Aggregating over replicate 'f" + str(fi) + "c" + str(ci) + "'")

        score_file = h5py.File(grad_dir + '/scores_f' + str(fi) + 'c' + str(ci) + '.h5', 'r')

        if rep_i == 0 :
            seqs = score_file['seqs'][()]
            grads = score_file['grads'][()]
            if 'preds' in score_file :
                preds = score_file['preds'][()]
            genes = score_file['gene'][()]
            chrs = score_file['chr'][()]
            starts = score_file['start'][()]
            ends = score_file['end'][()]
            strands = score_file['strand'][()]
        else :
            grads += score_file['grads'][()]
            if 'preds' in score_file :
                preds += score_file['preds'][()]

        #Collect garbage
        gc.collect()
        
        rep_i += 1

#Normalize by number of replicates
grads /= (float(len(fold_index)) * float(len(cross_index)))

if preds is not None :
    preds /= (float(len(fold_index)) * float(len(cross_index)))

#Re-save datasets in h5
scores_h5.create_dataset('seqs', data=np.array(seqs, dtype='bool'))
scores_h5.create_dataset('grads', data=np.array(grads, dtype='float16'))
if preds is not None :
    scores_h5.create_dataset('preds', data=np.array(preds, dtype='float16'))
scores_h5.create_dataset('gene', data=np.array(genes, dtype='S'))
scores_h5.create_dataset('chr', data=np.array(chrs, dtype='S'))
scores_h5.create_dataset('start', data=np.array(starts))
scores_h5.create_dataset('end', data=np.array(ends))
scores_h5.create_dataset('strand', data=np.array(strands, dtype='S'))

#Close h5
scores_h5.close()
